<a href="https://colab.research.google.com/github/vitorpq/LearnDataScience/blob/master/GA_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Parte 01**

Problema de seleção de atributos 

O dataset tem 413 instâncias ou linhas e 18937 atributos

(Fonte: https://www.openml.org/d/1122)

## Importar as bibliotecas

In [18]:
import numpy as np
import pandas as pd
import random
#import matplotlib.pyplot
#%matplotlib inline
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
# Preprocessing
from sklearn import preprocessing

## Carregar Dataset

In [28]:
#import the breast cancer dataset 
cancer=load_breast_cancer()
df = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
label=cancer["target"]

# 
#splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    label, test_size=0.30, 
                                                    random_state=101)
scaler = preprocessing.StandardScaler().fit(X_train)
X_sc_Train = scaler.transform(X_train)
scaler = preprocessing.StandardScaler().fit(X_test)
X_sc_Test = scaler.transform(X_test)
print(X_test)

     mean radius  mean texture  ...  worst symmetry  worst fractal dimension
107        12.36         18.54  ...          0.2983                  0.07185
437        14.04         15.98  ...          0.2725                  0.07234
195        12.91         16.33  ...          0.3024                  0.06949
141        16.11         18.05  ...          0.2792                  0.08158
319        12.43         17.00  ...          0.1901                  0.05932
..           ...           ...  ...             ...                      ...
420        11.57         19.04  ...          0.3035                  0.08284
279        13.85         15.18  ...          0.2955                  0.06912
390        10.26         12.22  ...          0.2937                  0.07722
91         15.37         22.76  ...          0.2556                  0.06828
297        11.76         18.14  ...          0.1978                  0.06915

[171 rows x 30 columns]


## Treinamento usando Regressão Logística

In [62]:
#training a logistics regression model
logmodel = LogisticRegression(max_iter=5000)
logmodel.fit(X_sc_Train,y_train)
predictions = logmodel.predict(X_sc_Test)
print("Accuracy = "+ str(accuracy_score(y_test,predictions)))

Accuracy = 0.9824561403508771


## GA

Now, let’s include a genetic algorithm in the process:
Defining all the steps required to follow during Genetic algorithm.

In [63]:
#defining various steps required for the genetic algorithm
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],y_train)
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,Xtrain,
                                   Xtest, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [64]:
chromo,score=generations(size=200,n_feat=30,n_parents=100,mutation_rate=0.10,
                     n_gen=38,Xtrain=X_sc_Train,Xtest=X_sc_Test,y_train=y_train,y_test=y_test)

[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9590643274853801, 0.9590643274853801]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9532163742690059, 0.9532163742690059]
[0.9532163742690059, 0.9532163742690059]
[0.9532163742690059, 0.9532163742690059]
[0.9532163742690059, 0.9532163742690059]
[0.9473684210526315, 0.9473684210526315]
[0.9532163742690059, 0.9532163742690059]
[0.9532163742690059, 0.9532163742690059]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9532163742690059, 0.9532163742690059]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526315, 0.9473684210526315]
[0.9473684210526

In [1]:
logmodel.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = logmodel.predict(X_test.iloc[:,chromo[-1]])
print("Accuracy score after genetic algorithm is= "+str(accuracy_score(y_test,predictions)))

NameError: ignored